# **Notebook de test de Téléchargement de donnée de basket depuis le site nba.com**

Sommaire

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
from TelechargementDonnees import (JourneeSiteNba, PasDeMatchError, JoueursSiteNba, 
                                   DriverFirefox, gererCookie, simplifierNomJoueur)
import pandas as pd
import numpy as np
import time
from collections import Counter

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import Select

## 1. Telecharger les données liees aux matchs depuis le site internet pour une ou plusieurs dates  
1. on telecharge une journee
1. on sauvegarder dans un json

In [10]:
#telecharger une journee
try : 
    journee=JourneeSiteNba('2021-01-26', dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
    #exporter
    #journee.saveCsv()
except PasDeMatchError as e : 
    print(e)

0 https://www.nba.com/game/lac-vs-atl-0022000268/box-score#box-score
1 https://www.nba.com/game/was-vs-hou-0022000269/box-score#box-score
2 https://www.nba.com/game/nyk-vs-uta-0022000270/box-score#box-score


In [14]:
# pour un ensemble de journee : 
for j in ['2021-01-%02d' %n  for n in range(25,27)] : 
    print(j)
    try :
        journee=JourneeSiteNba(j,dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
        journee.saveCsv()
    except PasDeMatchError as e: 
        print(e)    
        continue

2021-01-25
0 https://www.nba.com/game/phi-vs-det-0022000257/box-score#box-score
1 https://www.nba.com/game/tor-vs-ind-0022000258/box-score#box-score
2 https://www.nba.com/game/cha-vs-orl-0022000259/box-score#box-score
3 https://www.nba.com/game/mia-vs-bkn-0022000260/box-score#box-score
4 https://www.nba.com/game/lal-vs-cle-0022000261/box-score#box-score
5 https://www.nba.com/game/den-vs-dal-0022000265/box-score#box-score
6 https://www.nba.com/game/bos-vs-chi-0022000262/box-score#box-score
7 https://www.nba.com/game/min-vs-gsw-0022000266/box-score#box-score
8 https://www.nba.com/game/okc-vs-por-0022000267/box-score#box-score
2021-01-26
0 https://www.nba.com/game/lac-vs-atl-0022000268/box-score#box-score
1 https://www.nba.com/game/was-vs-hou-0022000269/box-score#box-score
2 https://www.nba.com/game/nyk-vs-uta-0022000270/box-score#box-score


## 2. Telecharger les données liees aux joueurs depuis le site internet 
> Dans le cas ou on souhaite telecharger la totalite des donnees de la page des joueuers
1. on se connecte à la page principale des joueurs
1. on affiche l'integralite des donnees

In [62]:
#creation de la classe
pagejoueuer=EnsembleJoueursSiteNba()
#acces aux donnees
pagejoueuer.dfJoueurs
#export des donnees
pagejoueuer.dfJoueurs.reset_index(drop=True).to_json(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\temp\basket\Joueurs_saison_2020-2021\Joueurs_saison_2020-2021.json')

## 3. Test donnees blessure sur site americain
1. on se connecte à la page des blessures par equipes pour recuperer tous les jouuers blesses et mettre a jour la table existante 
1. on se connecte à l apage des joueurs blesses par jour on prend que le plus recent.

Dans les deux cas on cherche les joueurs non presents dans la base des blesses et on fait des insert, ou on chreche les joueurs presents dans la base avec une date de guerison inconnues et on met a jour le type de blessures.

In [2]:
#se connecter au site
driver=DriverFirefox('Mano').driver
driver.get('https://www.cbssports.com/nba/injuries')
time.sleep(5)

In [3]:
#recuperer tous les joueurs blesse
importPage=pd.read_html(driver.page_source)
dfsInjuries=pd.concat([importPage[i] for i in range(len(importPage))])

In [4]:
#mettre a jour les nom
dfsInjuries['Player']=dfsInjuries.Player.apply(lambda x : ' '.join(x.split()[-2:]) if Counter(x)[' ']<=4 
                         and x.split()[-1].lower() not in ('jr.', 'sr.', 'ii','iii','iv','v')  
                         else ' '.join(x.split()[-3:])).tolist()
dfsInjuries['nom_simple']=dfsInjuries.Player.apply(lambda x : simplifierNomJoueur(x))

In [5]:
#récupérer les blesses en cours existants dans la base
with ct.ConnexionBdd('basket') as c :
    rqtBlesseEnCoursBdd="""SELECT b.*, j.nom, j.nom_simple
                             FROM donnees_source.blessure b JOIN donnees_source.joueur j ON b.id_joueur=j.id_joueur
                             WHERE date_guerison IS NULL"""
    blesseEnCoursBdd=pd.read_sql(rqtBlesseEnCoursBdd, c.sqlAlchemyConn)
#mettre à jour

NameError: name 'ct' is not defined